In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os
from PIL import Image
import numpy as np
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error


class ResNetEncoder(nn.Module):
    def __init__(self, num_channels=1):
        super(ResNetEncoder, self).__init__()

        # Define ResNet layers (modify based on your specific ResNet architecture)
        self.resnet = nn.Sequential(
            nn.Conv2d(num_channels, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            # Add more ResNet layers here...
        )

    def forward(self, x):
        x = self.resnet(x)
        return x

class FNNReducer(nn.Module):
    def __init__(self, input_size):  # Pass the input size here
        super(FNNReducer, self).__init__()

        # Define fully connected layers
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=input_size, out_features=1024),  # Use input_size
            nn.ReLU(inplace=True),
            nn.Linear(1024, 20 * 20),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.fc(x)
        return x

class CNNExpander(nn.Module):
    def __init__(self):
        super(CNNExpander, self).__init__()

        # Define convolutional layers
        self.conv_transpose = nn.ConvTranspose2d(
        in_channels=1,
        out_channels=1,
        kernel_size=(24, 24),
        stride=(24, 24),
        padding=(0, 0)
    )


    def forward(self, x):
        x = self.conv_transpose(x)
        return x

class UpsampleNet(nn.Module):
    def __init__(self):
        super(UpsampleNet, self).__init__()
        # Define the first ConvTranspose2d layer
        self.conv_transpose1 = nn.ConvTranspose2d(
            in_channels=1,
            out_channels=1,
            kernel_size=3,
            stride=2,
            padding=1,
            output_padding=1
        )
        # Define the second ConvTranspose2d layer
        self.conv_transpose2 = nn.ConvTranspose2d(
            in_channels=1,
            out_channels=1,
            kernel_size=3,
            stride=2,
            padding=1,
            output_padding=1
        )
        # ... Add more layers as needed ...
        # Define the final ConvTranspose2d layer to reach the desired size
        self.conv_transpose_final = nn.ConvTranspose2d(
            in_channels=1,
            out_channels=1,
            kernel_size=3,
            stride=24,
            padding=1
        )

    def forward(self, x):
        # Apply the first ConvTranspose2d layer
        x = self.conv_transpose1(x)
        # Apply the second ConvTranspose2d layer
        x = self.conv_transpose2(x)
        # ... Apply additional layers as needed ...
        # Apply the final ConvTranspose2d layer
        x = self.conv_transpose_final(x)
        return x

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.resnet_encoder = ResNetEncoder()
        # Calculate the input size for FNNReducer after a forward pass through ResNetEncoder
        dummy_input = torch.randn(1, 1, 480, 480)
        dummy_output = self.resnet_encoder(dummy_input)
        # Fix: Calculate input size considering the concatenation of two ResNet outputs
        input_size = dummy_output.shape[1] * 2 * dummy_output.shape[2] * dummy_output.shape[3]
        self.fnn_reducer = FNNReducer(input_size)  # Pass the calculated input size
        self.cnn_expander = CNNExpander()
        self.UpsampleNet= UpsampleNet()

    def forward(self, x1, x2):
        # ResNet encoding
        x1 = self.resnet_encoder(x1)
        x2 = self.resnet_encoder(x2)

        # Concatenate features
        x = torch.cat((x1, x2), dim=1)  # Concatenation doubles the number of channels

        # FNN reduction
        x = self.fnn_reducer(x)

        # Reshape to [batch_size, 1, 20, 20]
        x = x.view(-1, 1, 20, 20)
        #print(x.shape)
        # CNN expansion
        x = self.cnn_expander(x)

        return x

class ImageDataset(Dataset):
    def __init__(self, input1_folder, input2_folder, output_folder, transform=None):
        self.input1_folder = input1_folder
        self.input2_folder = input2_folder
        self.output_folder = output_folder
        self.input1_filenames = sorted(os.listdir(input1_folder))
        self.input2_filenames = sorted(os.listdir(input2_folder))
        self.output_filenames = sorted(os.listdir(output_folder))
        self.transform = transform

    def __len__(self):
        return len(self.input1_filenames)

    def __getitem__(self, idx):
        input1_path = os.path.join(self.input1_folder, self.input1_filenames[idx])
        input2_path = os.path.join(self.input2_folder, self.input2_filenames[idx])
        output_path = os.path.join(self.output_folder, self.output_filenames[idx])

        input1 = Image.open(input1_path).convert('L')
        input2 = Image.open(input2_path).convert('L')
        output = Image.open(output_path).convert('L')

        if self.transform:
            input1 = self.transform(input1)
            input2 = self.transform(input2)
            output = self.transform(output)

        return input1, input2, output

def train(model, train_dataloader, test_dataloader, optimizer, criterion, epochs):
# CUDA Usage:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = Model().to(device)  # Move the model to the GPU

    for epoch in range(epochs):
        running_loss = 0.0
        model.train()  # Set the model to training mode
        for i, data in enumerate(train_dataloader):
            input1, input2, target = data
            input1 = input1.to(device)
            input2 = input2.to(device)
            target = target.to(device)

            optimizer.zero_grad()

            # Forward pass
            output = model(input1, input2)

            # Calculate loss
            loss = criterion(output, target)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if i % 100 == 4:  # Print statistics every 4 batches
                print(f'[Epoch:{epoch + 1}, Batch:{i + 1:5d}] loss: {running_loss / 100:.3f}')
                running_loss = 0.0

             # Evaluation phase (after each epoch)
            model.eval()  # Set the model to evaluation mode
            with torch.no_grad():
                test_loss = 0.0
                all_predictions = []
                all_targets = []
                for input1, input2, target in test_dataloader:
                    input1 = input1.to(device)
                    input2 = input2.to(device)
                    target = target.to(device)

                    # Predict using the model
                    output = model(input1, input2)

                    # Calculate the loss
                    loss = criterion(output, target)
                    test_loss += loss.item()

                    # Store predictions and targets for MAPE calculation
                    all_predictions.append(output.cpu().numpy())
                    all_targets.append(target.cpu().numpy())

                # Calculate MAPE
                all_predictions = np.concatenate(all_predictions).flatten() # Flatten the array
                all_targets = np.concatenate(all_targets).flatten() # Flatten the array
                mape = mean_absolute_percentage_error(all_targets, all_predictions)

                # Print evaluation results
                if i % 100 == 4:  # Print statistics every 4 batches
                  print(f'[Epoch:{epoch + 1}] Test Loss: {test_loss / len(test_dataloader):.3f}, MAPE: {mape:.3f}')


# Define the training function (same as before)
# ... (code for train)



# Set paths to your input and output folders
input1_folder = '/content/drive/MyDrive/traindata/landfilll/input/EC_0.5'  # Replace with your actual path
input2_folder = '/content/drive/MyDrive/traindata/landfilll/input/thermal'  # Replace with your actual path
output_folder = '/content/drive/MyDrive/traindata/landfilll/label' # Replace with your actual path

# Define data transformations (e.g., normalization, resizing)
transform = transforms.Compose([
    transforms.Resize((480, 480)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Create the dataset
dataset = ImageDataset(input1_folder, input2_folder, output_folder, transform=transform)

# Split the dataset into train and test sets
train_dataset, test_dataset = train_test_split(dataset, test_size=0.1, random_state=42)

# Create dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)  # Use batch size 1 for test


# Create the model, optimizer, and loss function
model = Model()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Train the model
train(model, train_dataloader, test_dataloader, optimizer, criterion, epochs=500)

'''#Example usage:
model = Model()







# Input images (assume these are your 2-channel images)
image1 = torch.randn(1, 1, 480, 480)
image2 = torch.randn(1, 1, 480, 480)

# Pass images through the model
output = model(image1, image2)

# Output will have shape [1, 3, 480, 480]
print(output.shape)'''

[Epoch:1, Batch:    5] loss: 0.031
[Epoch:1] Test Loss: 0.615, MAPE: 1.025
[Epoch:2, Batch:    5] loss: 0.031
[Epoch:2] Test Loss: 0.615, MAPE: 1.025
[Epoch:3, Batch:    5] loss: 0.030
[Epoch:3] Test Loss: 0.615, MAPE: 1.025
[Epoch:4, Batch:    5] loss: 0.026
[Epoch:4] Test Loss: 0.615, MAPE: 1.025
[Epoch:5, Batch:    5] loss: 0.027
[Epoch:5] Test Loss: 0.615, MAPE: 1.025
[Epoch:6, Batch:    5] loss: 0.025
[Epoch:6] Test Loss: 0.615, MAPE: 1.025
[Epoch:7, Batch:    5] loss: 0.031
[Epoch:7] Test Loss: 0.615, MAPE: 1.025
[Epoch:8, Batch:    5] loss: 0.028
[Epoch:8] Test Loss: 0.615, MAPE: 1.025
[Epoch:9, Batch:    5] loss: 0.028
[Epoch:9] Test Loss: 0.615, MAPE: 1.025
[Epoch:10, Batch:    5] loss: 0.027
[Epoch:10] Test Loss: 0.615, MAPE: 1.025
[Epoch:11, Batch:    5] loss: 0.031
[Epoch:11] Test Loss: 0.615, MAPE: 1.025
[Epoch:12, Batch:    5] loss: 0.027
[Epoch:12] Test Loss: 0.615, MAPE: 1.025
[Epoch:13, Batch:    5] loss: 0.027
[Epoch:13] Test Loss: 0.615, MAPE: 1.025
[Epoch:14, Batch:

KeyboardInterrupt: 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
